In [1]:
IS_RENDER = 0
TCP_PORT = 12001

In [2]:
from mujoco_py import load_model_from_xml, load_model_from_path, MjSim, MjViewer
import math
import os
import numpy as np

from mujoco_py.modder import TextureModder

class manipulator():
    def __init__(self):
        
        self.DoF = DoF = 6
        self.model = load_model_from_path("model-ur3/ur3-camera.xml")
        self.sim = MjSim(self.model)
        
        self.sim.data.qpos[1] = -np.pi/2         
        self.sim.data.qpos[3] = -np.pi/2
        self.sim.step()

        if IS_RENDER: self.viewer = MjViewer(self.sim)
        self.sim_state = self.sim.get_state()
        
        self.state_dim = DoF*3+3
        self.action_dim = DoF
        
    def reset(self):
        self.sim.set_state(self.sim_state)
        
        rxyz = ("0.5 1.0 0.65").split(" ")
        
        self.rx, self.ry, self.rz = float(rxyz[0]), float(rxyz[1]), float(rxyz[2])
        
        self.qvel=np.zeros( self.DoF )
        
        s = []
        for i in range(self.DoF):
            s.append( self.qvel[i] )
        for i in range(self.DoF):
            s.append( self.sim.data.qvel[i] )
        for i in range(self.DoF):
            s.append( self.sim.data.qpos[i] )
        s.append( self.rx )
        s.append( self.ry )
        s.append( self.rz )
        s = np.array(s)
        
        return s
    def step(self, a):
                
        for i in range( self.DoF ):
            self.qvel[ i ] += a[i]
            
            # The limitation of velocity is important
            if self.qvel[i]>1.0:
                self.qvel[i] = 1.0
            if self.qvel[i]<-1.0:
                self.qvel[i] = -1.0
                
            self.sim.data.qvel[i] = self.qvel[i] 
                        

        
        self.sim.step()
        if IS_RENDER: self.viewer.render()
        
        R_mat = np.matrix([self.sim.data.sensordata[3:6], self.sim.data.sensordata[6:9], self.sim.data.sensordata[9:12]])    
        T = np.matrix([self.sim.data.sensordata[:3]])
        p_world = np.matrix([ [ self.rx, self.ry, self.rz ] ])

        p_end = (p_world - T)*(R_mat.T)   
        self.p_end = p_end
        p_camera = [ p_end[0,1], p_end[0,2], p_end[0,0] ]
        f = 0.01
        x_im = p_camera[0]*f/p_camera[2]
        y_im = p_camera[1]*f/p_camera[2]

        
        
        dis = np.linalg.norm( np.array( [x_im, y_im] ) - np.array( [0, 0] ) )
        r = -dis
        
        s = []
        for i in range(self.DoF):
            s.append( self.qvel[i] )
        for i in range(self.DoF):
            s.append( self.sim.data.qvel[i] )
        for i in range(self.DoF):
            s.append( self.sim.data.qpos[i] )
        s.append( self.rx )
        s.append( self.ry )
        s.append( self.rz )
        s = np.array(s)
        
        d = 0
        info = [dis]
        return s, r, d, info

## 1.2 Core Code of Servicer:

In [3]:
import numpy as np

import socket
import threading

import json

def env_servicer(env):
    #Create The Socket
    s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)

    #Listen The Port
    s.bind(('',TCP_PORT))
    s.listen(5)
    print( 'TCP_PORT: ', TCP_PORT, ',', 'Waiting for connection...')

    def tcplink(sock,addr):
        print('Accept new connection from %s:%s...' % addr)
        sock.send( ('Welcome!').encode() )
        while True:
            data=sock.recv(1024).decode()
            # print("data: ", data)

            data_json = json.loads( data )
            # print('data_json["type"]: ', data_json["type"])

            if data_json["type"] == "init":
                data = { 'state_dim' : env.state_dim, 'action_dim' : env.action_dim, 'DoF' : env.DoF } 

            elif data_json["type"] == "reset":
                state = env.reset()
                data = { 'state' : state.tolist() } 

            elif data_json["type"] == "step":
                a = np.array( data_json["action"] )
                state_next, r, done, info = env.step(a)
                data = { 'state' : state_next.tolist(), 'reward' : r, 'done' : done, 'info' : info } 
            elif data_json["type"] == "close":
                break
                
            str_json = json.dumps(data)
            sock.send( str_json.encode() )
        sock.close()
        np.save("sensordata_keep", sensordata_keep)
        np.save("qpos_keep", qpos_keep)

        print('Connection from %s:%s closed.'%addr)
        
    while True:
        # Accept A New Connection
        sock,addr=s.accept()
        
        # Create A New Thread to Deal with The TCP Connection
        t=threading.Thread(target=tcplink(sock,addr))

In [ ]:
env_servicer( manipulator() )

TCP_PORT:  12001 , Waiting for connection...
Accept new connection from 127.0.0.1:13936...


#### Analysis 002

##### Rotation

In [ ]:
env = manipulator()
env.reset()
self = env
R = np.matrix([self.sim.data.sensordata[3:6], self.sim.data.sensordata[6:9], self.sim.data.sensordata[9:12]])
T = np.matrix([self.sim.data.sensordata[:3]])
self.sim.data.sensordata[-3:]
rxyz = ("-0.15638756 -0.2379 -0.673").split(" ")
pos_end = np.matrix([ [float(rxyz[0]), float(rxyz[1]), float(rxyz[2])] ])
pos_end*R + T
p_world = np.matrix([[ 0.2379 ,  0.05555,  0.02115]])
(p_world - T)*(R.T)


#### Analysis 001

In [1]:
import numpy as np

In [5]:
rxyz = np.array([ -0.2986, 0.19425, 0.39555 ])

In [2]:
sensordata_keep = np.load( "sensordata_keep.npy" )

In [3]:
sensordata_keep.shape

(500, 3)

In [4]:
sensordata_keep[0]

array([ -3.60139680e-13,   1.94250000e-01,   6.94150000e-01])

In [6]:
np.linalg.norm( sensordata_keep[0] - rxyz )

0.42228416972435151

In [7]:
np.linalg.norm( sensordata_keep[-1] - rxyz )

0.056349188617117803

In [8]:
sensordata_keep[-1]

array([-0.35309129,  0.18174517,  0.40258987])